# Stacking

For: NEIL HEINRICH BRAUN

The idea for this file is to train a lightGBM model (or other models) for stacking. The data files you will need to import is unfortunately not ready. For now, imagine a file in the following format

| ticker | quarter_year | actual_log_rev | actual_car | log_revenue_prediction_1 | CAR_prediction_1 | log_revenue_prediction_2 | CAR_prediction_2 |
| ------ | ------------ | -------------- | ---------- | ------------------------ | ---------------- | ------------------------ | ---------------- |
| BAC    | Q1 2001      | 123            | 0.4        | 123                      | 0.5              | 123                      | 0.5              |
| JPM    | Q1 2001      | 123            | 0.4        | 456                      | 0.8              | 456                      | 0.8              |
| WFC    | Q1 2001      | 123            | 0.4        | 789                      | 0.25             | 789                      | 0.25             |

There will be more columns where each column will be the prediction from each previous model for each y value.

You will need to train at least 2 models for stacking. 1 for each y value. Of course try other things outside of LightGBM as well but I suspect GBM to be the best. If you try other things outside of GBM, process the data accordingly to get a good range.Your output should be in the format:

| ticker | quarter_year | log_revenue_prediction | CAR_prediction |
| ------ | ------------ | ---------------------- | -------------- |
| BAC    | Q1 2001      | 123                    | 0.5            |
| JPM    | Q1 2001      | 456                    | 0.8            |
| WFC    | Q1 2001      | 789                    | 0.25           |

Enjoy!


In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib
import os
from typing import List, Dict, Tuple, Union, Optional

warnings.filterwarnings("ignore")

# Create directory for models if it doesn't exist
os.makedirs('models', exist_ok=True)

In [ ]:
def load_data(filepath: str) -> pd.DataFrame:
    """
    Load the stacking data from a CSV file
    
    Args:
        filepath: Path to the CSV file
        
    Returns:
        Loaded DataFrame
    """
    # Load the data
    df = pd.read_csv(filepath)
    
    # Print some basic information about the data
    print(f"DataFrame shape: {df.shape}")
    print(f"Number of unique companies: {df['ticker'].nunique()}")
    print(f"Time period: {df['quarter_year'].min()} to {df['quarter_year'].max()}")
    
    # Create a numerical time_id from quarter_year for time-based splitting
    df['quarter'] = df['quarter_year'].str.extract(r'Q(\d)').astype(int)
    df['year'] = df['quarter_year'].str.extract(r'(\d{4})').astype(int)
    df['time_id'] = df['year'] * 4 + df['quarter']
    
    return df

In [ ]:
def prepare_features_for_stacking(
    df: pd.DataFrame, 
    target_column: str, 
    id_columns: List[str] = ['ticker', 'quarter_year', 'time_id', 'quarter', 'year']
) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Prepare features for stacking model
    
    Args:
        df: Input DataFrame with predictions from base models
        target_column: Name of the target column (actual values)
        id_columns: List of identifier columns to exclude from features
        
    Returns:
        X and y for training
    """
    # Filter columns that contain predictions (feature columns)
    feature_columns = [
        col for col in df.columns 
        if col not in id_columns and col != target_column and col != 'actual_log_rev' and col != 'actual_car'
    ]
    
    # Create feature matrix
    X = df[feature_columns].copy()
    
    # Create target vector
    y = df[target_column].copy()
    
    print(f"Features for stacking: {feature_columns}")
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    
    return X, y

In [ ]:
def time_series_cv(df: pd.DataFrame, n_splits: int = 3) -> List[Tuple[np.ndarray, np.ndarray]]:
    """
    Create time-based cross-validation splits
    
    Args:
        df: DataFrame to split
        n_splits: Number of splits (reduced to 3 for smaller dataset)
        
    Returns:
        List of train and validation indices
    """
    # Sort by time
    df = df.sort_values('time_id')
    
    # Create time-based splits with reduced number of splits
    tscv = TimeSeriesSplit(n_splits=n_splits)
    splits = []
    
    for train_idx, val_idx in tscv.split(df):
        splits.append((train_idx, val_idx))
    
    return splits

In [ ]:
def train_lightgbm_stacking_model(
    X: pd.DataFrame, 
    y: pd.Series, 
    cv_splits: List[Tuple[np.ndarray, np.ndarray]], 
    params: Optional[Dict] = None
) -> Tuple[lgb.Booster, float]:
    """
    Train a LightGBM stacking model with cross-validation
    
    Args:
        X: Feature matrix
        y: Target variable
        cv_splits: Cross-validation splits
        params: LightGBM parameters
        
    Returns:
        Trained model and average validation score
    """
    if params is None:
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'num_leaves': 7,  # Reduced for smaller dataset
            'learning_rate': 0.05,
            'feature_fraction': 0.8,
            'verbose': -1
        }
    
    # Keep track of validation scores
    val_scores = []
    fold_models = []
    
    # Train model with each fold
    for i, (train_idx, val_idx) in enumerate(cv_splits):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        # Create LightGBM datasets
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        
        # Train model
        callbacks = [
            lgb.early_stopping(stopping_rounds=20, verbose=False),  # Reduced for smaller dataset
            lgb.log_evaluation(period=0)  # Disable verbose output
        ]
        
        model = lgb.train(
            params, 
            train_data,
            valid_sets=[train_data, val_data],
            valid_names=['train', 'valid'],
            num_boost_round=200,  # Reduced for smaller dataset
            callbacks=callbacks
        )
        
        # Make predictions
        val_preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        r2 = r2_score(y_val, val_preds)
        
        val_scores.append(rmse)
        fold_models.append(model)
        
        print(f"Fold {i+1}/{len(cv_splits)}: RMSE = {rmse:.4f}, R² = {r2:.4f}")
    
    # Find the best model
    best_idx = np.argmin(val_scores)
    best_model = fold_models[best_idx]
    
    # Calculate average validation score
    avg_rmse = np.mean(val_scores)
    print(f"Average RMSE: {avg_rmse:.4f}")
    
    # Plot feature importance for the best model
    try:
        feature_importance = best_model.feature_importance()
        feature_names = X.columns.tolist()
        
        # Create a DataFrame for the feature importance
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': feature_importance
        }).sort_values('Importance', ascending=False)
        
        # Plot feature importance
        plt.figure(figsize=(10, 6))
        plt.barh(importance_df['Feature'], importance_df['Importance'])
        plt.title(f'Feature Importance (Best Model)')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.savefig(f'feature_importance.png')
        plt.close()
    except Exception as e:
        print(f"Could not plot feature importance: {e}")
    
    return best_model, avg_rmse

In [ ]:
def train_alternative_stacking_models(
    X: pd.DataFrame, 
    y: pd.Series, 
    cv_splits: List[Tuple[np.ndarray, np.ndarray]]
) -> Dict[str, Tuple[object, float]]:
    """
    Train alternative stacking models
    
    Args:
        X: Feature matrix
        y: Target variable
        cv_splits: Cross-validation splits
        
    Returns:
        Dictionary of trained models and their scores
    """
    # Define models to try (with parameters optimized for smaller datasets)
    models = {
        'RandomForest': RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=50, max_depth=3, random_state=42),
        'ElasticNet': ElasticNet(random_state=42, alpha=0.1, l1_ratio=0.5, max_iter=1000),
        'Ridge': Ridge(random_state=42, alpha=1.0, max_iter=1000)
    }
    
    # Store results
    results = {}
    
    # Standardize features for linear models
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    
    # Train each model
    for name, model in models.items():
        print(f"\nTraining {name}...")
        val_scores = []
        
        try:
            for i, (train_idx, val_idx) in enumerate(cv_splits):
                if name in ['ElasticNet', 'Ridge']:
                    # Use scaled features for linear models
                    X_train, y_train = X_scaled.iloc[train_idx], y.iloc[train_idx]
                    X_val, y_val = X_scaled.iloc[val_idx], y.iloc[val_idx]
                else:
                    # Use original features for tree-based models
                    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
                    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
                
                # Train model
                model.fit(X_train, y_train)
                
                # Make predictions
                val_preds = model.predict(X_val)
                rmse = np.sqrt(mean_squared_error(y_val, val_preds))
                r2 = r2_score(y_val, val_preds)
                
                val_scores.append(rmse)
                print(f"Fold {i+1}/{len(cv_splits)}: RMSE = {rmse:.4f}, R² = {r2:.4f}")
            
            # Calculate average validation score
            avg_rmse = np.mean(val_scores)
            print(f"Average RMSE: {avg_rmse:.4f}")
            
            # Store the model and its score
            if name in ['ElasticNet', 'Ridge']:
                # Also store the scaler for linear models
                results[name] = (model, avg_rmse, scaler)
            else:
                results[name] = (model, avg_rmse)
            
            # Try to plot feature importance if available
            try:
                if hasattr(model, 'feature_importances_'):
                    importance = model.feature_importances_
                    feature_names = X.columns.tolist()
                    
                    # Create a DataFrame for the feature importance
                    importance_df = pd.DataFrame({
                        'Feature': feature_names,
                        'Importance': importance
                    }).sort_values('Importance', ascending=False)
                    
                    # Plot feature importance
                    plt.figure(figsize=(10, 6))
                    plt.barh(importance_df['Feature'], importance_df['Importance'])
                    plt.title(f'Feature Importance ({name})')
                    plt.xlabel('Importance')
                    plt.tight_layout()
                    plt.savefig(f'feature_importance_{name}.png')
                    plt.close()
            except Exception as e:
                print(f"Could not plot feature importance for {name}: {e}")
        
        except Exception as e:
            print(f"Error training {name}: {e}")
            continue
    
    return results

In [ ]:
def find_best_model(lightgbm_rmse: float, alternative_models: Dict[str, Tuple[object, float]]) -> Tuple[str, object, float]:
    """
    Find the best model based on RMSE
    
    Args:
        lightgbm_rmse: RMSE for LightGBM model
        alternative_models: Dictionary of alternative models and their scores
        
    Returns:
        Name of the best model, the model itself, and its RMSE
    """
    # Compare LightGBM with alternative models
    best_rmse = lightgbm_rmse
    best_model_name = 'LightGBM'
    
    for name, model_info in alternative_models.items():
        model_rmse = model_info[1]
        if model_rmse < best_rmse:
            best_rmse = model_rmse
            best_model_name = name
    
    # Return the best model
    if best_model_name == 'LightGBM':
        print(f"\nBest model: {best_model_name} (RMSE: {best_rmse:.4f})")
        return best_model_name, None, best_rmse
    else:
        model = alternative_models[best_model_name][0]
        print(f"\nBest model: {best_model_name} (RMSE: {best_rmse:.4f})")
        return best_model_name, model, best_rmse

In [ ]:
def stacking_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    """
    Run the full stacking pipeline
    
    Args:
        df: Input DataFrame with predictions from base models
        
    Returns:
        DataFrame with stacked predictions
    """
    # Create time-based CV splits - do this once and reuse
    cv_splits = time_series_cv(df, n_splits=3)  # Reduced to 3 splits for smaller dataset
    
    # Split the data into revenue and CAR prediction tasks
    print("\n=== Revenue Stacking ===")
    
    # Prepare features for revenue prediction
    X_rev, y_rev = prepare_features_for_stacking(df, 'actual_log_rev')
    
    # Train LightGBM stacking model for revenue
    lightgbm_rev, lightgbm_rmse_rev = train_lightgbm_stacking_model(X_rev, y_rev, cv_splits)
    
    # Save the LightGBM model
    joblib.dump(lightgbm_rev, 'models/stacking_lightgbm_revenue.pkl')
    
    # Train alternative models for revenue
    alternative_models_rev = train_alternative_stacking_models(X_rev, y_rev, cv_splits)
    
    # Find the best model for revenue
    best_model_name_rev, best_model_rev, best_rmse_rev = find_best_model(lightgbm_rmse_rev, alternative_models_rev)
    
    # Save the best alternative model if it's better than LightGBM
    if best_model_name_rev != 'LightGBM' and best_model_rev is not None:
        joblib.dump(best_model_rev, f'models/stacking_{best_model_name_rev.lower()}_revenue.pkl')
    
    # Repeat for CAR prediction
    print("\n=== CAR Stacking ===")
    
    # Prepare features for CAR prediction
    X_car, y_car = prepare_features_for_stacking(df, 'actual_car')
    
    # Train LightGBM stacking model for CAR
    lightgbm_car, lightgbm_rmse_car = train_lightgbm_stacking_model(X_car, y_car, cv_splits)
    
    # Save the LightGBM model
    joblib.dump(lightgbm_car, 'models/stacking_lightgbm_car.pkl')
    
    # Train alternative models for CAR
    alternative_models_car = train_alternative_stacking_models(X_car, y_car, cv_splits)
    
    # Find the best model for CAR
    best_model_name_car, best_model_car, best_rmse_car = find_best_model(lightgbm_rmse_car, alternative_models_car)
    
    # Save the best alternative model if it's better than LightGBM
    if best_model_name_car != 'LightGBM' and best_model_car is not None:
        joblib.dump(best_model_car, f'models/stacking_{best_model_name_car.lower()}_car.pkl')
    
    # Generate final predictions
    # Use LightGBM models as default, override with better model if available
    df['log_revenue_prediction'] = lightgbm_rev.predict(X_rev)
    df['CAR_prediction'] = lightgbm_car.predict(X_car)
    
    # If a better model was found for revenue, use it instead
    if best_model_name_rev != 'LightGBM' and best_model_rev is not None:
        if best_model_name_rev in ['ElasticNet', 'Ridge']:
            # Need to scale the features for linear models
            scaler = alternative_models_rev[best_model_name_rev][2]
            X_rev_scaled = scaler.transform(X_rev)
            df['log_revenue_prediction'] = best_model_rev.predict(X_rev_scaled)
        else:
            df['log_revenue_prediction'] = best_model_rev.predict(X_rev)
    
    # If a better model was found for CAR, use it instead
    if best_model_name_car != 'LightGBM' and best_model_car is not None:
        if best_model_name_car in ['ElasticNet', 'Ridge']:
            # Need to scale the features for linear models
            scaler = alternative_models_car[best_model_name_car][2]
            X_car_scaled = scaler.transform(X_car)
            df['CAR_prediction'] = best_model_car.predict(X_car_scaled)
        else:
            df['CAR_prediction'] = best_model_car.predict(X_car)
    
    # Return final predictions in the required format
    final_predictions = df[['ticker', 'quarter_year', 'log_revenue_prediction', 'CAR_prediction']].copy()
    return final_predictions

In [ ]:
def main():
    """
    Main function to run the stacking pipeline
    """
    try:
        # Load the small stacking data
        df = load_data('data/stacking_data_expanded_small.csv')
        
        # Run the stacking pipeline
        predictions = stacking_pipeline(df)
        
        # Save the predictions
        predictions.to_csv('stacking_predictions.csv', index=False)
        
        print("\nStacking pipeline completed successfully!")
        print("Predictions saved to 'stacking_predictions.csv'")
        
        # Display a sample of the predictions
        print("\nSample predictions:")
        print(predictions.head(10))
        
        return predictions
    
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()
        return None

if __name__ == "__main__":
    # Run the main pipeline
    final_predictions = main()

## New Stacking

### Getting the train data

In [14]:
light_gbm_rev = pd.read_csv("data/stacking_data/lightgbm_rev_predict.csv")
neural_network_rev = pd.read_csv("data/stacking_data/neural_network_rev_predict.csv")
# ridge_reg_rev = pd.read_csv("data/stacking_data/ridge_regression_rev_predict.csv")  # Not ready!

# random_forest_car = pd.read_csv("data/stacking_data/random_forest_car_predict.csv")  # Not ready
neural_network_car = pd.read_csv("data/stacking_data/neural_network_car_predict.csv")
# lasso_reg_rev = pd.read_csv("data/stacking_data/lasso_regression_car_predict.csv")  # Not ready!


In [15]:
rev_truth = pd.read_csv("data/train_data_REV_with_text.csv")[["tic", "datacqtr", "Total Current Operating Revenue"]]
car_truth = pd.read_csv("data/train_data_CAR5_with_text.csv")[["tic", "datacqtr", "car5"]]

In [ ]:
df_rev_train = light_gbm_rev.merge(neural_network_rev, on=["tic", "datacqtr"], how="left")
# df_rev_train = df_rev_train.merge(ridge_reg_rev, on=["tic", "datacqtr"], how="left")
df_rev_train = df_rev_train.merge(rev_truth, on=["tic", "datacqtr"], how="left")

# df_car_train = random_forest_car.merge(neural_network_car, on=["tic", "datacqtr"], how="left")
# df_car_train = df_car_train.merge(lasso_reg_rev, on=["tic", "datacqtr"], how="left")
# df_car_train = df_car_train.merge(car_truth, on=["tic", "datacqtr"], how="left")

In [12]:
df_rev_train

,tic,datacqtr,light_gbm_rev_predict,neural_network_rev_predict,Total Current Operating Revenue
0,ALNC,2004Q1,0.240080,0.203927,0.240002
1,BNCN,2004Q1,0.179139,0.168254,0.176390
2,FCEC,2004Q1,0.241144,0.229981,0.231761
3,FSGI,2004Q1,0.227631,0.236375,0.221885
4,LNBB,2004Q1,0.238706,0.229413,0.236601
...,...,...,...,...,...
7271,WBS,2020Q4,0.539014,0.537056,0.537176
7272,WFC,2020Q4,0.932093,0.921146,0.919079
7273,WSBC,2020Q4,0.481852,0.490778,0.476202
7274,WSFS,2020Q4,0.482712,0.486162,0.482977


### Getting the test data

In [18]:
light_gbm_rev_test = pd.read_csv("data/results/lightgbm_rev_predict_test.csv")
neural_network_rev_test = pd.read_csv("data/results/neural_network_rev_predict_test.csv")
# ridge_reg_rev_test = pd.read_csv("data/stacking_data/ridge_regression_rev_predict.csv")  # Not ready!

# random_forest_car_test = pd.read_csv("data/stacking_data/random_forest_car_predict.csv")  # Not ready
neural_network_car_test = pd.read_csv("data/results/neural_network_car_predict_test.csv")
# lasso_reg_rev_test = pd.read_csv("data/stacking_data/lasso_regression_car_predict.csv")  # Not ready!

In [19]:
rev_truth_test = pd.read_csv("data/test_data_REV_with_text.csv")[["tic", "datacqtr", "Total Current Operating Revenue"]]
car_truth_test = pd.read_csv("data/test_data_CAR5_with_text.csv")[["tic", "datacqtr", "car5"]]

In [20]:
df_rev_test = light_gbm_rev_test.merge(neural_network_rev_test, on=["tic", "datacqtr"], how="left")
# df_rev_test = df_rev_test.merge(ridge_reg_rev_test, on=["tic", "datacqtr"], how="left")
df_rev_test = df_rev_test.merge(rev_truth_test, on=["tic", "datacqtr"], how="left")

# df_car_test = random_forest_car_test.merge(neural_network_car_test, on=["tic", "datacqtr"], how="left")
# df_car_test = df_car_test.merge(lasso_reg_rev_test, on=["tic", "datacqtr"], how="left")
# df_car_test = df_car_test.merge(car_truth_test, on=["tic", "datacqtr"], how="left")

In [21]:
df_rev_test

,tic,datacqtr,light_gbm_rev_predict,neural_network_rev_predict,Total Current Operating Revenue
0,ALRS,2022Q1,0.378146,0.329012,0.371809
1,AMAL,2022Q1,0.367589,0.364090,0.379766
2,AMNB,2022Q1,0.326019,0.311942,0.311230
3,AMTB,2022Q1,0.449265,0.437286,0.408613
4,AROW,2022Q1,0.347872,0.326976,0.339710
...,...,...,...,...,...
1016,WBS,2024Q4,0.646862,0.673665,0.654917
1017,WFC,2024Q4,0.966183,0.981408,0.964598
1018,WSBC,2024Q4,0.514048,0.537854,0.516255
1019,WSFS,2024Q4,0.549673,0.582713,0.546712
